In [7]:
import os

def find_pdf_file(directory, file_name):
    """
    Search for a PDF file with the specified name in the given directory and its subdirectories.

    :param directory: The root directory to start the search from.
    :param file_name: The name of the PDF file to search for.
    :return: The path to the PDF file if found, else None.
    """
    for root, dirs, files in os.walk(directory):
        if file_name in files:
            return os.path.join(root, file_name)
    return None

# Example usage
directory_to_search = '/Users/divyanshshukla/Downloads'
pdf_file_name = 'UG-Manual.pdf'
file_path = find_pdf_file(directory_to_search, pdf_file_name)

if file_path:
    print(f'File found: {file_path}')
else:
    print('File not found')


File not found


In [8]:
!pip install PyMuPDF



In [4]:
!pip install faiss-cpu



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 51.5 MB/s eta 0:00:00


In [5]:
!pip install sentence-transformers



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [9]:
import fitz
import faiss
from sentence_transformers import SentenceTransformer
pdf_path = '/content/UG-Manual.pdf'
def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            text += page.get_text()
        return text
    except Exception as e:
        print(f"Error opening PDF: {e}")
        return ""
pdf_text = extract_text_from_pdf(pdf_path)
if pdf_text:
    print("Text extracted successfully.")
else:
    print("Failed to extract text.")

def chunk_text(text, chunk_size=200):
    words = text.split()
    chunks = [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

chunks = chunk_text(pdf_text)

model = SentenceTransformer('all-MiniLM-L6-v2')

chunk_embeddings = model.encode(chunks, convert_to_tensor=True)

d = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(d)

index.add(chunk_embeddings.cpu().numpy())

def retrieve_relevant_chunk(query, model, index, chunks, top_k=1):
    query_embedding = model.encode([query], convert_to_tensor=True).cpu().numpy()
    distances, indices = index.search(query_embedding, top_k)
    relevant_chunks = [chunks[idx] for idx in indices[0]]
    return relevant_chunks

def pdf_chatbot(pdf_path, query):
    pdf_text = extract_text_from_pdf(pdf_path)

    chunks = chunk_text(pdf_text)

    chunk_embeddings = model.encode(chunks, convert_to_tensor=True)
    d = chunk_embeddings.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(chunk_embeddings.cpu().numpy())
    relevant_chunks = retrieve_relevant_chunk(query, model, index, chunks)
    return relevant_chunks

query = "Explain branch change."
relevant_text = pdf_chatbot(pdf_path, query)
print(relevant_text)


Text extracted successfully.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


['of branch / programme for a student is a matter of privilege and not a right. Once a Branch Change has been granted, a student cannot revert to the original department. 10.1.1 Eligibility A student may apply after her/his second semester if she/he has acquired credits (i.e., received a passing grade) for all the course prescribed in the template for the first year of the programme, including the courses where S/X grades are awarded. Any student may apply after their third or fourth semester even if they have NOT acquired credits (i.e., received a passing grade) for all the courses prescribed in the template for the first year of the programme, including the courses where S/X grades are awarded. 10.1.2 Application Process i. The DoAA office will call for branch change applications twice in an academic year in April and November. ii. Eligible students may apply on the prescribed branch change form addressed to the Chairperson, SUGC. iii. Completed application forms should be submitted 